In [1]:
# Import python libraries
#
import importlib
import pickle
import os
import torch

# Go one directory back, because all imports are done
# relative to the root of the project.
#
project_root = '..'
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    os.chdir(project_root)

# Imports own modules.
#
import scripts.Visualization as Visualization
import scripts.ModelTrainer as ModelTrainer
import scripts.Utils as utils
import scripts.Simulation_config as Simulation_config
from scripts.Simulation_config import *
import scripts.Model as model
import scripts.ModelAdapter as ModelAdapter


/home/molu/miniconda3/envs/xlstm/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/home/molu/miniconda3/envs/xlstm'


In [ ]:
importlib.reload(model)
importlib.reload(ModelTrainer)
importlib.reload(Simulation_config)
importlib.reload(utils)

# Run the whole simulation
#
train_all_models = False
if train_all_models:
    configs = Simulation_config.configs
    ModelTrainer.ModelTrainer().run(configs)

# Print the sumarized simulation results
#
utils.Evaluate_Models.print_results('scripts/outputs/all_train_histories.pkl')


("Configuration: Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_50households_each.pkl', "
 'nrOfComunities=20, trainingHistory=365, modelInputHistory=24, '
 "usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction', 'xLSTM', "
 "'LSTM', 'Transformer'), epochs=100)")
Latex Summary for this Configuration:  & 32.97 (4.37) & 14.41 (0.45) & 12.92 (0.62) & 10.48 (0.48) & 10.76 (0.66) & 11.23 (0.80)
("Configuration: Config_of_one_run(modelSize='SMALL', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_50households_each.pkl', "
 'nrOfComunities=20, trainingHistory=365, modelInputHistory=24, '
 "usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction', 'xLSTM', "
 "'LSTM', 'Transformer'), epochs=100)")
Latex Summary for this Configuration:  & 32.97 (4.37) & 14.41 (0.45) & 12.92 (0.62) & 11.00 (0.55) & 10.68 (0.58) & 10.90 (0.64)
("Configurati

defaultdict(<function scripts.Utils.Evaluate_Models.print_results.<locals>.<lambda>()>,
            {})

In [32]:
# Plot the losses over the training.
# (To see, if the models are trained stable.)
#
importlib.reload(utils)

plot_only_single_config = True
plotted_config = Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, 
    NrOfComunities._20, TrainingHistory._12_MONTH, ModelInputHistory._1_DAY, UsedModels.ALL, Epochs.DEFAULT)

utils.Evaluate_Models.plot_training_losses_over_epochs('scripts/outputs/all_train_histories.pkl',
                                                       plot_only_single_config,
                                                       plotted_config
                                                       )


Plotted Config:
("Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_50households_each.pkl', "
 'nrOfComunities=20, trainingHistory=365, modelInputHistory=24, '
 "usedModels=('SyntheticLoadProfile', 'KNN', 'PersistencePrediction', 'xLSTM', "
 "'LSTM', 'Transformer'), epochs=100)")


In [6]:
# Evaluate specific model of given power profiles and given configurations
#

importlib.reload(Visualization)
importlib.reload(model)
importlib.reload(utils)
importlib.reload(ModelAdapter)

# Define a specific configuration
#
test_profile = "scripts/outputs/file_0.pkl"
with open(test_profile, 'rb') as f:
    (X, Y, modelAdapter) = pickle.load(f)
path_to_trained_parameters = 'scripts/outputs/all_trained_models.pth'
model_type = 'PersistencePrediction'
num_of_features = X['train'].shape[2]
myConfig = Config_of_one_run(ModelSize.MEDIUM, DoPretraining.YES, DoTransferLearning.YES, Aggregation_Count._50_HOUSEHOLDS, 
                NrOfComunities._20, TrainingHistory._2_MONTH, ModelInputHistory._1_DAY, UsedModels.ALL, Epochs.DEFAULT)

# Plot the chosen model
#
myModel = utils.Deserialize.get_trained_model(path_to_trained_parameters, model_type, test_profile, myConfig, num_of_features)
plotlyApp = Visualization.PlotlyApp(X, Y, myModel, modelAdapter, None, 'UTC')
plotlyApp.run(myport=8051)


In [ ]:
# Print the model parameter sizes
#

importlib.reload(model)

model_types = ['xLSTM', 'LSTM', 'Transformer']
for model_type in model_types:
    for model_size in ['SMALL', 'MEDIUM', 'LARGE']:

        # Test, if the model is run-able
        num_of_features = 18
        m = model.Model(model_type, model_size, num_of_features)
        x = torch.zeros((7, 24*22, num_of_features))
        m.my_model(x)

        # Print the model's parameter count
        nr_of_parameters = m.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


xLSTM - SMALL has 19791 parameters.
xLSTM - MEDIUM has 26411 parameters.
xLSTM - LARGE has 97679 parameters.
LSTM - SMALL has 17463 parameters.
LSTM - MEDIUM has 44031 parameters.
LSTM - LARGE has 104831 parameters.
Transformer - SMALL has 24423 parameters.
Transformer - MEDIUM has 50863 parameters.
Transformer - LARGE has 113351 parameters.
